**Loading All the Library Needed**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
from pathlib import Path
from sklearn.model_selection import train_test_split
import os.path

import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten

from tensorflow import keras

from sklearn.metrics import r2_score

# ICA

**Loading the Model**

In [ ]:
from keras.models import model_from_json
json_file = open('/content/drive/MyDrive/Data/Model/ResNet50/ICA/model.json', 'r')
model_json = json_file.read()
json_file.close()
# Loading the model
model_ica = model_from_json(model_json)
# Loading the weights
model_ica.load_weights('/content/drive/MyDrive/Data/Model/ResNet50/ICA/model.h5')
print('Model is successfully loaded from Drive')
print(model_ica.summary())


Model is successfully loaded from Drive
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 216, 144, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 5, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________
None


**Evaluation Looping for Each Version in Source 3**

In [ ]:
def evaluate(model, base, i):
  image_dir = Path('/content/drive/MyDrive/Data/CNN {}/source3/v{}'.format(base, i))
  filepaths = pd.Series(list(image_dir.glob(r'**/*.png')), name='Filepath').astype(str)
  bpms = pd.Series(filepaths.apply(lambda x: (os.path.split(os.path.split(x)[1])[1])[:-4]), name='BPMS').astype(np.float64)
  images = pd.concat([filepaths, bpms], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

  # Printing Info
  print("-----Currently Working on Version {}-----".format(i))
  print('Total Data: {}'.format(len(images)))

  # Building Test Generator
  test_image_df = images.sample(len(images), random_state = 1).reset_index(drop=True)
  test_generator = tf.keras.preprocessing.image.ImageDataGenerator()
  test_images_flow = test_generator.flow_from_dataframe(
    dataframe=test_image_df,
    x_col='Filepath',
    y_col='BPMS',
    target_size=(216, 144),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
  )

  predicted_bpm = np.squeeze(model.predict(test_images_flow))
  true_bpm = test_images_flow.labels

  # Getting MAE
  mae = np.mean(np.abs(predicted_bpm-true_bpm))
  print('MAE for Version {}: {}'.format(i, mae))

In [ ]:
for i in range(1,8):
  evaluate(model_ica, 'ICA', i)

-----Currently Working on Version 1-----
Total Data: 104
Found 104 validated image filenames.
MAE for Version 1: 9.082215682790396
-----Currently Working on Version 2-----
Total Data: 106
Found 106 validated image filenames.
MAE for Version 2: 8.922269029337262
-----Currently Working on Version 3-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 3: 8.495436463556759
-----Currently Working on Version 4-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 4: 8.529060150024444
-----Currently Working on Version 5-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 5: 8.663434029765222
-----Currently Working on Version 6-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 6: 8.540507026789756
-----Currently Working on Version 7-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 7: 20.007090793908684


# CHROM

**Loading the Model**

In [ ]:
from keras.models import model_from_json
json_file = open('/content/drive/MyDrive/Data/Model/ResNet50/CHROM/model.json', 'r')
model_json = json_file.read()
json_file.close()
# Loading the model
model_chrom = model_from_json(model_json)
# Loading the weights
model_chrom.load_weights('/content/drive/MyDrive/Data/Model/ResNet50/CHROM/model.h5')
print('Model is successfully loaded from Drive')
print(model_chrom.summary())

Model is successfully loaded from Drive
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 216, 144, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 5, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [ ]:
for i in range(1,8):
  evaluate(model_chrom, 'Chrom', i)

-----Currently Working on Version 1-----
Total Data: 102
Found 102 validated image filenames.
MAE for Version 1: 9.944390187371987
-----Currently Working on Version 2-----
Total Data: 106
Found 106 validated image filenames.
MAE for Version 2: 9.779602466836627
-----Currently Working on Version 3-----
Total Data: 106
Found 106 validated image filenames.
MAE for Version 3: 9.78047891152726
-----Currently Working on Version 4-----
Total Data: 101
Found 101 validated image filenames.
MAE for Version 4: 9.614909307913292
-----Currently Working on Version 5-----
Total Data: 100
Found 100 validated image filenames.
MAE for Version 5: 9.284984950267695
-----Currently Working on Version 6-----
Total Data: 101
Found 101 validated image filenames.
MAE for Version 6: 10.663767186893319
-----Currently Working on Version 7-----
Total Data: 103
Found 103 validated image filenames.
MAE for Version 7: 20.153807632567357


# PCA

**Loading the Model**

In [ ]:
json_file = open('/content/drive/MyDrive/Data/Model/ResNet50/PCA/model.json', 'r')
model_json = json_file.read()
json_file.close()
# Loading the model
model_pca = model_from_json(model_json)
# Loading the weights
model_pca.load_weights('/content/drive/MyDrive/Data/Model/ResNet50/PCA/model.h5')
print('Model is successfully loaded from Drive')
print(model_pca.summary())

Model is successfully loaded from Drive
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 216, 144, 3)]     0         
                                                                 
 resnet50 (Functional)       (None, 7, 5, 2048)        23587712  
                                                                 
 global_average_pooling2d (G  (None, 2048)             0         
 lobalAveragePooling2D)                                          
                                                                 
 dense (Dense)               (None, 1)                 2049      
                                                                 
Total params: 23,589,761
Trainable params: 2,049
Non-trainable params: 23,587,712
_________________________________________________________________
None


In [ ]:
for i in range(1,8):
  evaluate(model_pca, 'PCA', i)

-----Currently Working on Version 1-----
Total Data: 103
Found 103 validated image filenames.
MAE for Version 1: 9.57114319179824
-----Currently Working on Version 2-----
Total Data: 106
Found 106 validated image filenames.
MAE for Version 2: 9.36528295165355
-----Currently Working on Version 3-----
Total Data: 106
Found 106 validated image filenames.
MAE for Version 3: 8.475681589622344
-----Currently Working on Version 4-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 4: 8.136586739538894
-----Currently Working on Version 5-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 5: 9.220450787839253
-----Currently Working on Version 6-----
Total Data: 105
Found 105 validated image filenames.
MAE for Version 6: 8.45721107686777
-----Currently Working on Version 7-----
Total Data: 104
Found 104 validated image filenames.
MAE for Version 7: 20.295903612600856
